In [24]:
# input an age number (e.g. 150), and return a sorted list of time concepts in which this age number locates 
# this function only applies to the global vocabulary schemes, such as isc2012-08 
# this query takes about 10 seconds
install.packages("SPARQL")
library(SPARQL)

Installing package into ‘/nethome/max/.R-libs’
(as ‘lib’ is unspecified)



In [25]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"
  
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = " 
prefix dc: <http://purl.org/dc/elements/1.1/> 
prefix dcterms: <http://purl.org/dc/terms/> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix geo: <http://www.opengis.net/ont/geosparql#> 
prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#> 
prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rank: <http://resource.geosciml.org/ontology/timescale/rank/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix samfl: <http://def.seegrid.csiro.au/ontology/om/sam-lite#> 
prefix sf: <http://www.opengis.net/ont/sf#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix sosa: <http://www.w3.org/ns/sosa/> 
prefix thors: <http://resource.geosciml.org/ontology/timescale/thors#> 
prefix time: <http://www.w3.org/2006/time#> 
prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
prefix vann: <http://purl.org/vocab/vann/> 
prefix void: <http://rdfs.org/ns/void#> 
prefix xkos: <http://rdf-vocabulary.ddialliance.org/xkos#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix tsna: <http://deeptimekb.org/tsna#> 
prefix tssc: <http://deeptimekb.org/tssc#> 
prefix tsnc: <http://deeptimekb.org/tsnc#> 
prefix tsnz: <http://deeptimekb.org/tsnz#> 
prefix tswe: <http://deeptimekb.org/tswe#> 
prefix tsbr: <http://deeptimekb.org/tsbr#> 
prefix tsba: <http://deeptimekb.org/tsba#> 
prefix tsjp: <http://deeptimekb.org/tsjp#> 
prefix tsau: <http://deeptimekb.org/tsau#> 
        "

In [38]:
# This query will find the Age-level concept
    
q0 = paste0( sparql_prefix, '
SELECT DISTINCT ?tconcept  ?lbl
WHERE
{
  GRAPH <http://deeptimekb.org/iscallnew>
 {
    ?tconcept  a  gts:Age ; 
               skos:inScheme ts:isc2012-08; 
               rdfs:label ?lbl;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ts:isc2012-08
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ts:isc2012-08
                   ] .

    FILTER (?baseNum >= 150)
    FILTER (?topNum <= 150) 
  }
}
ORDER BY ?baseNum
LIMIT 1
 '
     )
res0 = SPARQL(endpoint, q0)$results

In [41]:
# This query will find and list all the concepts at different levels
# We avoid using nested query to save time
    
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT ?All ?cLabelEn
WHERE
{
GRAPH <http://deeptimekb.org/iscallnew>
{

{ #This block is to include the Age-level concept in the resulting list 
  ', res0$tconcept[1], ' rdfs:label ?labelAgeC . 
  ?All rdfs:label ?labelAgeC .
}
UNION
{ # This and the next block are to find the broader and broaderTransitive concepts
', res0$tconcept[1], '
dc:description
      [
         skos:broaderTransitive ?All ;
         skos:inScheme  ts:isc2012-08
      ]
}
UNION 
{
', res0$tconcept[1], '
dc:description
      [
         skos:broader  ?All ;
         skos:inScheme  ts:isc2012-08
      ]

}
  
      ?All 
      rdfs:label ?cLabelEn ;
      dc:description 
      [  
        time:hasBeginning ?baseboundary1 ;
        time:hasEnd ?topboundary1 ;
        skos:inScheme ts:isc2012-08
        ] .
      
      ?baseboundary1 time:inTemporalPosition ?basetime1 .
      ?topboundary1 time:inTemporalPosition ?toptime1 .
      
      ?basetime1 dc:description
      [
        time:numericPosition ?baseNum1;
        skos:inScheme ts:isc2012-08
        ]  .
      
      ?toptime1 dc:description
      [
        time:numericPosition ?topNum1;
        skos:inScheme ts:isc2012-08
        ]  .
      
    }
}

ORDER BY (?baseNum1 - ?topNum1)
                  '
              )
# run the query  
res = SPARQL(endpoint, q)$results
  
print(res)



                                                                  All
1     <http://resource.geosciml.org/classifier/ics/ischart/Tithonian>
2 <http://resource.geosciml.org/classifier/ics/ischart/UpperJurassic>
3      <http://resource.geosciml.org/classifier/ics/ischart/Jurassic>
4      <http://resource.geosciml.org/classifier/ics/ischart/Mesozoic>
5   <http://resource.geosciml.org/classifier/ics/ischart/Phanerozoic>
                   cLabelEn
1        "Tithonian Age"@en
2 "Upper Jurassic Epoch"@en
3      "Jurassic Period"@en
4         "Mesozoic Era"@en
5      "Phanerozoic Eon"@en


In [42]:
#Below is another solution to find the list of concepts at different levels 
ConceptLevels <- c("Age", "Epoch", "Sub-Period", "Period", "Era", "Eon", "Super-Eon")

cLevelLength = length(ConceptLevels)

cat("In ts:isc2012-08","\n")

for (j in 1:cLevelLength)  {  
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT ?tconcept  str(?lbl) AS ?cLabel
WHERE
{
    {
       ?tconcept  a gts:GeochronologicEra ,
                 gts:', ConceptLevels[j] , ';  
               skos:inScheme ts:isc2012-08 ; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition ;
               time:hasEnd ?topPosition. 
      ?basePosition time:inTemporalPosition ?baseTime .
      ?topPosition time:inTemporalPosition ?topTime .

      ?baseTime time:numericPosition ?baseNum .
      ?topTime time:numericPosition ?topNum .
    }
   UNION 
    { 
       ?tconcept  a gts:GeochronologicEra ,
                 gts:', ConceptLevels[j] , ';  
               skos:inScheme ts:isc2012-08; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ts:isc2012-08
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
     } 

    FILTER (?baseNum >= 150)
    FILTER (?topNum <= 150) 

}
ORDER BY ?baseNum
LIMIT 1
        '
          )
# run the query  
resConcepts = SPARQL(endpoint, q)$results

cat(ConceptLevels[j], "Level: ", resConcepts$cLabel[1], "\n")
    
}

In ts:isc2012-08 
Age Level:  Tithonian Age 
Epoch Level:  Upper Jurassic Epoch 
Sub-Period Level:  
Period Level:  Jurassic Period 
Era Level:  Mesozoic Era 
Eon Level:  Phanerozoic Eon 
Super-Eon Level:  
